In [190]:
import pandas as pd
import datetime
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score, mean_absolute_error
from sklearn.preprocessing import binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score ,roc_auc_score, explained_variance_score, mean_squared_error, r2_score, plot_roc_curve

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.9.0+cu111
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['seed', 'partition', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Pandas Datareader Test

In [191]:
total = pd.read_csv('./result/qsofa_4h.csv')
print(len(total))

214104


In [192]:
# print(len(total))
# temp = total[total['is_infection']==1]
# total = pd.concat([total, temp])
# total = pd.concat([total, temp])
# e = len(total[total['is_infection']==1])
# print(e*4/len(total))

In [193]:
total.columns

Index(['starttime', 'stay_id', 'hr', 'endtime', 'sbp', 'gcs_min',
       'respiratory_rate', 'bp_score', 'rr_score', 'gcs_score', 'qsofa',
       'event'],
      dtype='object')

In [194]:
total.columns
# print(type(total))
total['starttime'] = pd.to_datetime(total['starttime'])
total.replace({'event': True}, {'event': 1}, inplace=True)
total.replace({'event': False}, {'event': 0}, inplace=True)
total.dtype

Index(['starttime', 'stay_id', 'hr', 'endtime', 'sbp', 'gcs_min',
       'respiratory_rate', 'bp_score', 'rr_score', 'gcs_score', 'qsofa',
       'event'],
      dtype='object')

In [195]:
total.head(10)

,starttime,stay_id,hr,endtime,sbp,gcs_min,respiratory_rate,bp_score,rr_score,gcs_score,qsofa,event
0,2130-05-25 13:00:00,35258336,0,2130-05-25 17:00:00,100.025000,15.0,15.941667,0,0,0,0,0.0
1,2130-05-25 17:00:00,35258336,1,2130-05-25 21:00:00,108.875000,15.0,15.750000,0,0,0,0,0.0
2,2130-05-25 21:00:00,35258336,2,2130-05-26 01:00:00,108.250000,15.0,12.750000,0,0,0,0,0.0
3,2130-05-26 01:00:00,35258336,3,2130-05-26 05:00:00,102.750000,15.0,12.750000,0,0,0,0,0.0
4,2167-08-09 05:00:00,35938282,0,2167-08-09 09:00:00,111.666667,15.0,23.250000,0,1,0,1,0.0
5,2167-08-09 09:00:00,35938282,1,2167-08-09 13:00:00,108.250000,15.0,21.250000,0,0,0,0,0.0
6,2167-08-09 13:00:00,35938282,2,2167-08-09 17:00:00,114.166667,15.0,22.250000,0,1,0,1,0.0
7,2167-08-09 17:00:00,35938282,3,2167-08-09 21:00:00,120.500000,15.0,17.166667,0,0,0,0,0.0
8,2136-10-03 18:00:00,34582336,0,2136-10-03 22:00:00,162.375000,15.0,14.166667,0,0,0,0,0.0
9,2136-10-03 22:00:00,34582336,1,2136-10-04 02:00:00,160.250000,15.0,16.000000,0,0,0,0,0.0


In [196]:
total=total.set_index('starttime')
# total=total.sort_values(by=['Unnamed: 0'])
total = total.drop(['stay_id', 'hr', 'endtime', 'bp_score', 'rr_score', 'gcs_score', 'qsofa'], axis=1)
#total = total.astype('float')

In [197]:
total.columns
print(total.index)

DatetimeIndex(['2130-05-25 13:00:00', '2130-05-25 17:00:00',
               '2130-05-25 21:00:00', '2130-05-26 01:00:00',
               '2167-08-09 05:00:00', '2167-08-09 09:00:00',
               '2167-08-09 13:00:00', '2167-08-09 17:00:00',
               '2136-10-03 18:00:00', '2136-10-03 22:00:00',
               ...
               '2176-06-15 22:00:00', '2176-06-16 02:00:00',
               '2139-05-26 18:00:00', '2139-05-26 22:00:00',
               '2139-05-27 02:00:00', '2139-05-27 06:00:00',
               '2164-03-22 17:00:00', '2164-03-22 21:00:00',
               '2164-03-23 01:00:00', '2164-03-23 05:00:00'],
              dtype='datetime64[ns]', name='starttime', length=214104, freq=None)


In [198]:
print(total.values)

[[100.025       15.          15.94166667   0.        ]
 [108.875       15.          15.75         0.        ]
 [108.25        15.          12.75         0.        ]
 ...
 [107.5         15.          25.75         0.        ]
 [106.75        15.          26.5          0.        ]
 [105.75        15.          25.           0.        ]]


In [199]:
print(total.columns)

Index(['sbp', 'gcs_min', 'respiratory_rate', 'event'], dtype='object')


# Data Preparation

In [200]:
class SepsisDataset(torch.utils.data.Dataset):
    def __init__(self, df_data, seq_len=4):
        self.seq_len = seq_len
        self.X = df_data.loc[:,df_data.columns!='event'].values
        self.y = df_data.loc[:, 'event'].values
        
    def __len__(self):
        return len(self.X)//self.seq_len
    
    def __getitem__(self, idx):
        idx += self.seq_len # 몇 시간 단위로 인덱스 올리는 거
        X = self.X[idx-self.seq_len:idx-2]
        y = self.y[idx-1]
        return X, y 

In [201]:
train_dataset = SepsisDataset(total, seq_len=4)
trainloader = DataLoader(train_dataset,  # dataset을 input으로
                             batch_size=1000,
                             shuffle=True, drop_last=True)

for i, (X, y) in enumerate(trainloader) :
    print(X.size() , y.size())

torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.Size([1000])
torch.Size([1000, 2, 3]) torch.S

# Model Define

In [202]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = 1
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        # 나중에 network(node 개수 바꿔 주는 옵션도 생각하기)
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        layers.append(nn.Sigmoid())
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
      # Model Capacity 따라 y.sahpe이 달라짐
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1)) # 맨 마지막 h_t 갖고 예측 할거니까 -1
#         t = Variable(torch.Tensor([0.5]))  # threshold
#         out = (y_pred > t).float() * 1
        return y_pred

# Train, Validate, Test 

In [204]:
def train(model, partition, optimizer, loss_fn, args):
  # batch 단위로 갖고 오는 것
  # 무작위로 시작 record 갖고 옴
  # 갖고 올때 batch size만큼 갖고 옴
  # permutation, not random
    trainloader = DataLoader(partition['train'],  # dataset을 input으로
                             batch_size=args.batch_size,
                             shuffle=True, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    train_acc = 0.0
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):

        # X : [10, n, 6] [n, 10, 6] 현재 타임스템프 안에
        # Y : [10, m, 1] 
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y.float().to(args.device) 

        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += metric(y_pred, y_true)

    train_loss = train_loss / len(trainloader)
    train_acc = train_acc / len(trainloader)
    
    return model, train_loss, train_acc

In [205]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    val_acc = 0.0
    val_loss = 0.0
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y.float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            val_acc += metric(y_pred, y_true)

    val_loss = val_loss / len(valloader)
    val_acc = val_acc / len(valloader)
    return val_loss, val_acc

In [206]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    test_acc = 0.0
    cnt = 0
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            cnt += 1
#             print(cnt)
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y.float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            if cnt == 1:
                y_true_all = torch.tensor(y_true)
                y_pred_all = torch.tensor(y_pred)
            else:
                y_true_all = torch.cat([y_true_all, y_true], dim=0)
                y_pred_all = torch.cat([y_pred_all, y_pred], dim=0)
                
            test_acc += metric(y_pred, y_true)
        
    test_acc = test_acc / len(testloader)
#     print(len(y_pred_all), len(y_true_all))
    
    return y_pred, test_acc, y_true_all, y_pred_all

In [207]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)

    # loss_fn = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
    loss_fn = nn.BCELoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ===================================== #
    
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        model, train_loss, train_acc = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        if epoch % 10 == 0:
            print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    y_pred, test_acc, y_true, y_pred_all = test(model, partition, args)   
    print('len of temp:', len(temp))
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
#     result['pred'] = y_pred.cpu().detach().numpy()
    return vars(args), result, y_true, y_pred_all

# Manage Experiment

In [208]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [209]:

def plot_acc(var1, var2, df):

    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    sns.barplot(x=var1, y='test_acc', hue=var2, data=df, ax=ax[2])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    ax[2].set_title('Test Accuracy')

    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) 


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

In [210]:
train_val_dataset = total[:int((len(total) / 4)*.8)*4]
test_dataset = total[int((len(total) / 4)*.8)*4:]
train_dataset = train_val_dataset[:int((len(train_val_dataset) / 4)*.8)*4]
valid_dataset = train_val_dataset[int((len(train_val_dataset) / 4)*.8)*4:]

trainset = SepsisDataset(train_dataset, seq_len=4)
valset = SepsisDataset(valid_dataset, seq_len=4)
testset = SepsisDataset(test_dataset, seq_len=4)

print(len(test_dataset)/4)

partition = {'train': trainset, 'val':valset, 'test':testset}

10706.0


In [236]:
def metric(y_pred, y_true):  
#     auc = roc_auc_score(y_true, y_pred) 
    
    perc_y_pred = y_pred.cpu().detach().numpy()
    perc_y_true = y_true.cpu().detach().numpy()

    perc_y_pred = np.where(perc_y_pred > 0.5, 1, 0)
    perc_y_pred = np.squeeze(perc_y_pred)
    
#     auc = accuracy_score(perc_y_true, perc_y_pred, normalize=True)
    
#     print('pred:', perc_y_pred)
#     print('true:', perc_y_true)
#     auc = roc_auc_score(perc_y_true, perc_y_pred)
    try:
        auc = roc_auc_score(perc_y_true, perc_y_pred)
    except ValueError:
        print('Error')
        auc = 0.00

    return auc

In [237]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr"
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.batch_size = 64

# ====== Model Capacity ===== #
args.input_dim = 3
args.hid_dim = 60
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.00001
args.dropout = 0.0
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 50


# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'n_layers'
# list_var1 = [0.001, 0.0001, 0.00001]
list_var1 = [0.0001]
# list_var2 = [1,2,3]
list_var2 = [3]

for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result, y_true, y_pred_all = experiment(partition, deepcopy(args))
        save_exp_result(setting, result)

Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=60, input_dim=3, l2=1e-05, lr=0.0001, n_layers=3, optim='Adam', use_bn=True)
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Epoch 0, Acc(train/val): 0.49/0.50, Loss(train/val) 0.40360/0.25947. Took 3.84 sec
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


KeyboardInterrupt: 

# Evaluation

In [ ]:
var1 = 'lr'
var2 = 'n_layers'
df = load_exp_result('exp1')

plot_acc(var1, var2, df)
plot_loss_variation(var1, var2, df, sharey=False) #sharey를 True로 하면 모둔 subplot의 y축의 스케일이 같아집니다.
plot_acc_variation(var1, var2, df, margin_titles=True, sharey=True)

In [ ]:
y_pred_all = y_pred_all.cpu().detach().numpy()
y_true = y_true.cpu().detach().numpy()
y_pred_all = np.where(y_pred_all > 0.5, 1, 0)

print(len(y_pred_all))
print(len(y_true))

In [ ]:
acc = accuracy_score(y_true, y_pred_all, normalize=True)
auc = roc_auc_score(y_true, y_pred_all)
auc

In [ ]:
from sklearn.metrics import classification_report

t = classification_report(y_true, y_pred_all, target_names=['True', 'False'])

In [ ]:
print(t)

In [ ]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    tt = confusion[0,0]
    tf = confusion[0,1]
    ft = confusion[1,0]
    ff = confusion[1,1]
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='weighted') # None, 'micro', 'macro', 'weighted'
    recall = recall_score(y_test, y_pred,average='weighted')
    F1 = f1_score(y_test, y_pred,average='weighted')
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))    
    print('양성예측율: {:.4f}'.format(tt/(tt+tf)))
    print('음성예측율: {:.4f}'.format(ff/(ft+ff)))    
    print('F1: {:.4f}'.format(F1))

In [ ]:
get_clf_eval(y_true, y_pred_all)

In [ ]:
auc